In [4]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

from sodapy import Socrata

### Chicago
Data source for Chicago:

https://data.cityofchicago.org/Health-Human-Services/COVID-19-Daily-Cases-and-Deaths/naz8-j4nc

There is an API that I will use to generate just the date and the number of deaths, starting with February 1.

In [20]:
client = Socrata("data.cityofchicago.org", None)

In [21]:
results = client.get("naz8-j4nc", limit=2000)

In [25]:
results_df = pd.DataFrame.from_records(results)

In [87]:
results_df.dropna(inplace=True)

In [88]:
def get_days_since_feb_1(date_str):
    dt = datetime.strptime(date_str,
                           '%Y-%m-%dT%H:%M:%S.%f')
    return (dt - datetime(2020, 2, 1)).days

In [91]:
results_df['days_since_feb_1'] = results_df['lab_report_date'].apply(get_days_since_feb_1)

In [92]:
num_days = results_df['days_since_feb_1'].max()

In [105]:
results_df.set_index('days_since_feb_1', inplace=True)

In [114]:
def get_deaths(days_since_feb_1, df=results_df):
    if days_since_feb_1 in df.index:
        return int(df.loc[days_since_feb_1]['deaths_total'])
    else:
        return 0

In [119]:
index = [datetime(2020, 2, 1) + timedelta(days=days) for days in range(num_days+1)]
deaths = [get_deaths(days) for days in range(num_days+1)]

In [125]:
chi_death_df = pd.DataFrame(deaths,
                       index=index,
                       columns=['deaths'])

In [126]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')